In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
from sklearn import svm, datasets
import sklearn.model_selection as model_selection
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score


In [ ]:
pip install cvxpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np 
import matplotlib
import matplotlib.pyplot as plt
import random
import time
import cvxpy as cp

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/ECS271/Assignment3/studentspen-train.csv')

In [ ]:
df.head()

,x3,y3,x4,y4,x5,y5,x6,y6,Digit
0,10,75,0,24,48,0,100,19,0
1,16,69,0,27,40,0,93,15,0
2,11,5,63,0,100,43,89,99,0
3,47,0,86,25,100,66,76,100,0
4,57,0,100,22,95,75,54,100,0


In [ ]:
df.Digit.value_counts()

2    392
4    391
1    390
0    384
5    375
7    374
3    370
8    363
9    357
6    351
Name: Digit, dtype: int64

In [ ]:
X=df[['x3','y3','x4','y4','x5','y5','x6','y6']]

In [ ]:

df1=list(zip(*[X[col] for col in X]))
X=[list(i) for i in df1]


In [ ]:

Y=df['Digit'].to_list()
Y=np.array(Y)

In [ ]:

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X=scaler.fit_transform(X)

###One Vs One Predict function

In [ ]:
def predict(x):
  x=np.squeeze(np.asarray(x))
  # x=x.reshape((x.shape[0],1))
  vote=[0 for i in range(10)]

  for i in range(9):
    for j in range(i+1,10): 
      w=W_dic[(i,j)]
      b=B_dic[(i,j)]
      w=np.squeeze(np.asarray(w))
      y=np.dot(w,x)+b 
      # print('y == ', y)
      # print(j)
      if y>0:
        vote[i]+=1
      else:
        vote[j]+=1  
  mm=max(vote)

  return vote.index(mm)         



In [ ]:
# print('ans : ', y_test[2])
# predict(X_test[2])

###KFold Cross Validation

In [ ]:
import numpy as np
from sklearn.model_selection import KFold
kf = KFold(n_splits=5,shuffle=True,random_state=1)

In [ ]:
KFold_accuracies=[]
num_sv=[]
total_samples=[]
ns_by_n=[]

In [ ]:

for train_index, test_index in kf.split(X):
  # print("TRAIN:", train_index, "TEST:", test_index)
  X_train, X_test = X[train_index], X[test_index]
  y_train, y_test = Y[train_index], Y[test_index]


  X_train_dic={}

  for i in range(len(X_train)):
    if y_train[i] not in X_train_dic:
      X_train_dic[y_train[i]]=[X_train[i]]
    else:
      X_train_dic[y_train[i]].append(X_train[i])


  X_test_dic={}

  for i in range(len(X_test)):
    if y_test[i] not in X_test_dic:
      X_test_dic[y_test[i]]=[X_test[i]]
    else:
      X_test_dic[y_test[i]].append(X_test[i])     

  #ONE VS ONE MULTICLASS CLASSIFICATION

  W_dic={}
  B_dic={}
  eps = 1e-8

  ll=list(range(10))
  for i in range(9):
    for j in range(i+1,10):
      data1=np.array(X_train_dic[i])
      data2=np.array(X_train_dic[j])
      N1=data1.shape[0]
      N2=data2.shape[0]
      D=8
      xx = np.concatenate((data1, data2), axis=0)
      
      yy = np.concatenate((np.ones((N1, 1)), - np.ones((N2, 1))), axis=0)

      # USE THE SOLVER 

      C = 2
      w = cp.Variable((D, 1))
      b = cp.Variable()
      epsilon = cp.Variable((N1+N2, 1))
      objective = cp.Minimize(cp.sum(cp.square(w))*0.5 + cp.sum(cp.square(epsilon)*C))
      constraints = [cp.multiply(yy, (xx @ w + b)) >= 1 - epsilon,  epsilon >= 0]
      prob = cp.Problem(objective, constraints)
      prob.solve(max_iter=40000)
      W_dic[(i,j)]=w.value
      B_dic[(i,j)]=b.value 
      
      #Number of support vectors
      sup = yy*(xx.dot(w.value)+b.value)-1
      sup_v1 = ((-eps<sup) & (sup<eps)).flatten()
      num_sv.append(len(sup_v1))
      total_samples.append(len(X_train))
      

  #PREDICTING on the test data

  corr=0
  for i in range(len(y_test)):
    if y_test[i]==predict(X_test[i]):
      corr+=1
  acc=corr/len(y_test)    
  KFold_accuracies.append(acc)
  n_s=sum(num_sv)
  N=sum(total_samples)
  ns_by_n.append((n_s/N))
  num_sv=[]
  total_samples=[]

###Mean Accuracy on the K-Fold cross Validation

In [ ]:
KFold_accuracies=np.array(KFold_accuracies)
ns_by_n=np.array(ns_by_n)
theo_bound=ns_by_n.mean()
empirical_acc=KFold_accuracies.mean()
print(f'Theoretical error-rate(Ns/N) is {theo_bound}, while as the empirical accuracy achieved is {empirical_acc} ')

Theoretical error-rate(Ns/N) is 0.2, while as the empirical accuracy achieved is 0.8593520249221184 


###Student's Test.csv

In [ ]:
test_df=pd.read_csv('/content/drive/MyDrive/ECS271/Assignment3/studentsdigits-test.csv')
test_df.head()
X_test=test_df[['x3','y3','x4','y4','x5','y5','x6','y6']]
df1=list(zip(*[X_test[col] for col in X_test]))
X_test=[list(i) for i in df1]

X_test=scaler.transform(X_test)
ans=[]
for i in range(len(X_test)):
  ans.append(predict(X_test[i]))
      
ans=np.array(ans)
np.savetxt('/content/drive/MyDrive/ECS271/Assignment3/ans1.txt',ans)

#QUESTION 4

###Training the SVM on 1 vs 9

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/ECS271/Assignment3/studentspen-train.csv')
X=df[['x3','y3','x4','y4','x5','y5','x6','y6']]
df1=list(zip(*[X[col] for col in X]))
X=[list(i) for i in df1]
Y=df['Digit'].to_list()
Y=np.array(Y)

In [ ]:
from sklearn.preprocessing import StandardScaler
X=np.array(X)
scaler = StandardScaler()
X=scaler.fit_transform(X)

In [ ]:

X_transfer={}

for i in range(len(X)):
  if Y[i] not in X_transfer:
    X_transfer[Y[i]]=[X[i]]
  else:
    X_transfer[Y[i]].append(X[i])

X_transfer.keys()

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [ ]:
X_train=X_transfer[1] + X_transfer[9]
Y_train=[1 for i in range(len(X_transfer[1]))] + [-1 for i in range(len(X_transfer[9]))]

In [ ]:
N1=len(X_transfer[1])
N2=len(X_transfer[9])
print('Num. of 1s',len(X_transfer[1]),',  Num. of 9s',len(X_transfer[9]))

Num. of 1s 390 ,  Num. of 9s 357


In [ ]:
Y_train=np.array(Y_train).reshape(-1,1)
X_train=np.array(X_train)

In [ ]:
X_train.shape

(747, 8)

In [ ]:
# + cp.sum(cp.square(epsilon)*C)
D=8
C = 2
w = cp.Variable((D, 1))
b = cp.Variable()
epsilon = cp.Variable((N1+N2, 1))
objective = cp.Minimize(cp.sum(cp.square(w))*0.5 + cp.sum(cp.square(epsilon)*C) )
constraints = [cp.multiply(Y_train, (X_train @ w + b)) >= 1-epsilon, epsilon>=0]
prob = cp.Problem(objective, constraints)
prob.solve(max_iter=70000)

492.8277909232536

In [ ]:
def predict_transfer(x,w,b):
  x=np.squeeze(np.asarray(x))
  w=np.squeeze(np.asarray(w))
  
  y=np.dot(w,x)+b 
  # print('y == ', y)
  # print(j)
  if y>0:
    return 1
  else:
    return -1 
          

#return -1 for 9 or 7

In [ ]:
corr=0
for i in range(Y_train.shape[0]):
  if Y_train[i][0]==predict_transfer(X_train[i],w.value,b.value):
    corr+=1
print('Training accuracy for 1 vs 9 is : ',corr/(Y_train.shape[0]),corr)    


Training accuracy for 1 vs 9 is :  0.8902275769745649 665


##Accuracy on 1 vs 7 without any Transfer.
 

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/ECS271/Assignment3/transfer_test.csv')
X=df[['x3','y3','x4','y4','x5','y5','x6','y6']]
df1=list(zip(*[X[col] for col in X]))
X=[list(i) for i in df1]
Y=df['Digit'].to_list()

for i in range(len(Y)):
  if Y[i]==7:
    Y[i]=-1
xx=np.array(X)
yy=np.array(Y).reshape(-1,1)
# yy

In [ ]:
corr=0
for i in range(yy.shape[0]):
  if yy[i][0]==predict_transfer(xx[i],w.value,b.value):
    corr+=1
print('Accuracy for 1 vs 7 without any transfer is : ',corr/(yy.shape[0]),corr,'correct pred. out of yy.shape[0]')    


Accuracy for 1 vs 7 without any transfer is :  0.5102339181286549 349 correct pred. out of yy.shape[0]


In [ ]:
w_source=w.value
b_source=b.value

###Hypothesis Transfer

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/ECS271/Assignment3/transfer_train.csv')
X=df[['x3','y3','x4','y4','x5','y5','x6','y6']]
df1=list(zip(*[X[col] for col in X]))
X=[list(i) for i in df1]
Y=df['Digit'].to_list()

for i in range(len(Y)):
  if Y[i]==7:
    Y[i]=-1

In [ ]:
xx=np.array(X)
yy=np.array(Y).reshape(-1,1)
# yy
D=8
P = np.diag(np.ones(D))
w = cp.Variable((D, 1))
w.value = w_source
b = cp.Variable()
epsilon = cp.Variable((len(xx), 1))
objective = cp.Minimize(cp.sum_squares(w) * 0.5 + cp.sum(cp.square(epsilon)*C))
constraints = [cp.multiply(yy, (xx @ w + b)) >= 1-epsilon, cp.sum_squares((w_source - w)) <= 100, epsilon>=0 ]
prob = cp.Problem(objective, constraints)
prob.solve()

35.03853451209694

###Training Accuracy for Hypothesis Transfer


In [ ]:
corr=0
for i in range(yy.shape[0]):
  if yy[i][0]==predict_transfer(xx[i],w.value,b.value):
    corr+=1
print('Training accuracy for Hypothesis Transfer (1 vs 7) is : ',corr/(yy.shape[0]),' num. of correct predictions: ',corr,' out of ', yy.shape[0])    


Training accuracy for Hypothesis Transfer (1 vs 7) is :  0.9125  num. of correct predictions:  73  out of  80


In [ ]:
xx.shape

(80, 8)

###Testing Accuracy for Hypothesis Transfer

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/ECS271/Assignment3/transfer_test.csv')
X=df[['x3','y3','x4','y4','x5','y5','x6','y6']]
df1=list(zip(*[X[col] for col in X]))
X=[list(i) for i in df1]
Y=df['Digit'].to_list()

for i in range(len(Y)):
  if Y[i]==7:
    Y[i]=-1
xx=np.array(X)
yy=np.array(Y).reshape(-1,1)
# yy

In [ ]:
corr=0
for i in range(yy.shape[0]):
  if yy[i][0]==predict_transfer(xx[i],w.value,b.value):
    corr+=1
print('Testing accuracy for Hypothesis Transfer (1 vs 7) is : ',corr/(yy.shape[0]),' num. of correct predictions: ',corr,' out of ', yy.shape[0])    


Testing accuracy for Hypothesis Transfer (1 vs 7) is :  0.8961988304093568  num. of correct predictions:  613  out of  684


###Instance Transfer

In [ ]:
w_source,b_source

(array([[ 0.09985203],
        [ 0.74046131],
        [ 0.60504691],
        [ 0.22483394],
        [ 0.26570817],
        [-0.63586391],
        [-0.93017331],
        [ 0.39412601]]), array(-0.41404025))

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/ECS271/Assignment3/transfer_train.csv')
X=df[['x3','y3','x4','y4','x5','y5','x6','y6']]
df1=list(zip(*[X[col] for col in X]))
X=[list(i) for i in df1]
Y=df['Digit'].to_list()

X=np.array(X)
X=scaler.transform(X)

In [ ]:
for i in range(len(Y)):
  if Y[i]==7:
    Y[i]=-1

In [ ]:
xx=np.array(X)
yy=np.array(Y).reshape(-1,1)
# yy

####Distance of Each point in transfer_train.csv from the margin in the w_source as its weight

In [ ]:
Imp=yy*(np.dot(xx,w_source) + b_source) - 1
func = lambda t: 1/(abs(t[0])+0.000001)
weight = np.array([func(xi) for xi in Imp])
weight=weight.reshape(-1,1)

In [ ]:
weight

array([[36.19801744],
       [65.89909247],
       [ 2.8077099 ],
       [ 0.88951621],
       [ 2.95668804],
       [16.99376265],
       [ 2.14706658],
       [ 6.71138881],
       [ 4.73286814],
       [ 1.11537659],
       [ 1.26547756],
       [ 2.75956524],
       [ 1.33567425],
       [ 3.06979832],
       [ 7.6685752 ],
       [ 0.63143807],
       [10.41457622],
       [ 1.31776539],
       [ 1.42572188],
       [52.4494682 ],
       [16.10240618],
       [ 1.57398115],
       [ 1.01478893],
       [ 1.32002751],
       [ 1.942592  ],
       [ 1.57371582],
       [ 2.40219306],
       [ 1.40694851],
       [ 3.83862568],
       [ 1.56955555],
       [ 4.47020177],
       [ 0.91220481],
       [ 8.85572074],
       [ 4.33764378],
       [ 1.36948105],
       [ 1.75999008],
       [ 1.19917187],
       [12.11637817],
       [ 1.08780311],
       [ 1.69696768],
       [ 0.39563592],
       [ 0.35779874],
       [ 0.3400876 ],
       [ 0.4405149 ],
       [ 0.29014336],
       [ 0

In [ ]:
weight.shape

(80, 1)

In [ ]:
D=8
w = cp.Variable((D, 1))
b = cp.Variable()
epsilon = cp.Variable((len(xx), 1))
objective = cp.Minimize(cp.sum_squares(w) * 0.5 + cp.sum(cp.square(epsilon)*C))
constraints = [cp.multiply(cp.multiply(weight, yy), (xx @ w + b)) >= 1-epsilon, epsilon>=0]
prob = cp.Problem(objective, constraints)
prob.solve()

50.54760677399933

In [ ]:
corr=0
for i in range(yy.shape[0]):
  if yy[i][0]==predict_transfer(xx[i],w.value,b.value):
    corr+=1
print('Training accuracy for Instance Transfer (1 vs 7) is : ',corr/(yy.shape[0]),' num. of correct predictions: ',corr,' out of ', yy.shape[0])    


Training accuracy for Instance Transfer (1 vs 7) is :  0.9  num. of correct predictions:  72  out of  80


### Accuracy of Instance Transfer on transfer_test.csv

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/ECS271/Assignment3/transfer_test.csv')
X=df[['x3','y3','x4','y4','x5','y5','x6','y6']]
df1=list(zip(*[X[col] for col in X]))
X=[list(i) for i in df1]
Y=df['Digit'].to_list()


In [ ]:
for i in range(len(Y)):
  if Y[i]==7:
    Y[i]=-1

In [ ]:
xx=np.array(X)
yy=np.array(Y).reshape(-1,1)
# yy

In [ ]:
corr=0
for i in range(yy.shape[0]):
  if yy[i][0]==predict_transfer(xx[i],w.value,b.value):
    corr+=1
print('Testing accuracy for Instance Transfer (1 vs 7) is : ',corr/(yy.shape[0]),' num. of correct predictions: ',corr,' out of ', yy.shape[0])    


Testing accuracy for Instance Transfer (1 vs 7) is :  0.6067251461988304  num. of correct predictions:  415  out of  684


#BONUS

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/ECS271/Assignment3/studentspen-train.csv') 
X=df[['x3','y3','x4','y4','x5','y5','x6','y6']]
df1=list(zip(*[X[col] for col in X]))
X=[list(i) for i in df1]


Y=df['Digit'].to_list()
Y=np.array(Y)



from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X=scaler.fit_transform(X)
###new array for holding projected dimensions.
X_new=np.zeros((X.shape[0],46))

### Projecting onto the new dimensions using Polynomial Basis Fn.
###  (1 + x1 + x2 + x3 + ... + x8 )**2

for i in range(X.shape[0]):
  for j in range(8):
    X_new[i][j]=X[i][j]
  X_new[i][8]=X[i][0]*X[i][0]
  X_new[i][9]=X[i][1]*X[i][1]
  X_new[i][10]=X[i][2]*X[i][2]
  X_new[i][12]=X[i][3]*X[i][3]
  X_new[i][13]=X[i][4]*X[i][4]
  X_new[i][14]=X[i][5]*X[i][5]
  X_new[i][15]=X[i][6]*X[i][6]
  X_new[i][16]=X[i][7]*X[i][7]

  X_new[i][17]=2*X[i][0]*X[i][1]
  X_new[i][18]=2*X[i][0]*X[i][2]  
  X_new[i][19]=2*X[i][0]*X[i][3]  
  X_new[i][20]=2*X[i][0]*X[i][4]  
  X_new[i][21]=2*X[i][0]*X[i][5] 
  X_new[i][22]=2*X[i][0]*X[i][6]
  X_new[i][23]=2*X[i][0]*X[i][7]

  X_new[i][24]=2*X[i][1]*X[i][2]  
  X_new[i][25]=2*X[i][1]*X[i][3]  
  X_new[i][26]=2*X[i][1]*X[i][4]  
  X_new[i][27]=2*X[i][1]*X[i][5] 
  X_new[i][28]=2*X[i][1]*X[i][6]
  X_new[i][29]=2*X[i][1]*X[i][7]

  X_new[i][30]=2*X[i][2]*X[i][3]  
  X_new[i][31]=2*X[i][2]*X[i][4]  
  X_new[i][32]=2*X[i][2]*X[i][5] 
  X_new[i][33]=2*X[i][2]*X[i][6]
  X_new[i][34]=2*X[i][2]*X[i][7]

  X_new[i][35]=2*X[i][3]*X[i][4]  
  X_new[i][36]=2*X[i][3]*X[i][5] 
  X_new[i][37]=2*X[i][3]*X[i][6]
  X_new[i][38]=2*X[i][3]*X[i][7]

  X_new[i][39]=2*X[i][4]*X[i][5] 
  X_new[i][40]=2*X[i][4]*X[i][6]
  X_new[i][41]=2*X[i][4]*X[i][7]

  X_new[i][42]=2*X[i][5]*X[i][6]
  X_new[i][43]=2*X[i][5]*X[i][7]

  X_new[i][44]=2*X[i][6]*X[i][7]

  X_new[i][45]=1

X_train, X_test, y_train, y_test = model_selection.train_test_split(X_new, Y, train_size=0.80, test_size=0.20, random_state=101)


X_train_dic={}

for i in range(len(X_train)):
  if y_train[i] not in X_train_dic:
    X_train_dic[y_train[i]]=[X_train[i]]
  else:
    X_train_dic[y_train[i]].append(X_train[i])

X_test_dic={}
for i in range(len(X_test)):
  if y_test[i] not in X_test_dic:
    X_test_dic[y_test[i]]=[X_test[i]]
  else:
    X_test_dic[y_test[i]].append(X_test[i])

# data1=np.array(X_train_dic[0])

W_dic={}
B_dic={}

ll=list(range(10))
for i in range(9):
  for j in range(i+1,10):
    data1=np.array(X_train_dic[i])
    data2=np.array(X_train_dic[j])
    N1=data1.shape[0]
    N2=data2.shape[0]
    D=46
    X = np.concatenate((data1, data2), axis=0)
    
    y = np.concatenate((np.ones((N1, 1)), - np.ones((N2, 1))), axis=0)

    # USE THE SOLVER 

    C = 2
    w = cp.Variable((D, 1))
    b = cp.Variable()
    epsilon = cp.Variable((N1+N2, 1))
    objective = cp.Minimize(cp.sum(cp.square(w))*0.5 + cp.sum(cp.square(epsilon)*C))
    constraints = [cp.multiply(y, (X @ w + b)) >= 1 - epsilon,  epsilon >= 0]
    prob = cp.Problem(objective, constraints)
    prob.solve(max_iter=40000)
    W_dic[(i,j)]=w.value
    B_dic[(i,j)]=b.value




In [ ]:
def predict(x):
  x=np.squeeze(np.asarray(x))
  # x=x.reshape((x.shape[0],1))
  vote=[0 for i in range(10)]

  for i in range(9):
    for j in range(i+1,10): 
      w=W_dic[(i,j)]
      b=B_dic[(i,j)]
      w=np.squeeze(np.asarray(w))
      y=np.dot(w,x)+b 
      # print('y == ', y)
      # print(j)
      if y>0:
        vote[i]+=1
      else:
        vote[j]+=1  
  mm=max(vote)

  return vote.index(mm)         



In [ ]:
corr=0
for i in range(X_test.shape[0]): 
  if y_test[i]==predict(X_test[i]):
    corr+=1 
print(corr/X_test.shape[0])  
print('Testing accuracy for BONUS question(kernel) is : ', corr/X_test.shape[0])  

0.968
Testing accuracy for BONUS question(kernel) is :  0.968


##Prediction on Student's Digits-Test

In [ ]:
test_df=pd.read_csv('/content/drive/MyDrive/ECS271/Assignment3/studentsdigits-test.csv')
test_df.head()
X=test_df[['x3','y3','x4','y4','x5','y5','x6','y6']]
df1=list(zip(*[X[col] for col in X]))
X=[list(i) for i in df1]

X=scaler.transform(X)


X_new=np.zeros((X.shape[0],46))

for i in range(X.shape[0]):
  for j in range(8):
    X_new[i][j]=X[i][j]
  X_new[i][8]=X[i][0]*X[i][0]
  X_new[i][9]=X[i][1]*X[i][1]
  X_new[i][10]=X[i][2]*X[i][2]
  X_new[i][12]=X[i][3]*X[i][3]
  X_new[i][13]=X[i][4]*X[i][4]
  X_new[i][14]=X[i][5]*X[i][5]
  X_new[i][15]=X[i][6]*X[i][6]
  X_new[i][16]=X[i][7]*X[i][7]

  X_new[i][17]=2*X[i][0]*X[i][1]
  X_new[i][18]=2*X[i][0]*X[i][2]  
  X_new[i][19]=2*X[i][0]*X[i][3]  
  X_new[i][20]=2*X[i][0]*X[i][4]  
  X_new[i][21]=2*X[i][0]*X[i][5] 
  X_new[i][22]=2*X[i][0]*X[i][6]
  X_new[i][23]=2*X[i][0]*X[i][7]

  X_new[i][24]=2*X[i][1]*X[i][2]  
  X_new[i][25]=2*X[i][1]*X[i][3]  
  X_new[i][26]=2*X[i][1]*X[i][4]  
  X_new[i][27]=2*X[i][1]*X[i][5] 
  X_new[i][28]=2*X[i][1]*X[i][6]
  X_new[i][29]=2*X[i][1]*X[i][7]

  X_new[i][30]=2*X[i][2]*X[i][3]  
  X_new[i][31]=2*X[i][2]*X[i][4]  
  X_new[i][32]=2*X[i][2]*X[i][5] 
  X_new[i][33]=2*X[i][2]*X[i][6]
  X_new[i][34]=2*X[i][2]*X[i][7]

  X_new[i][35]=2*X[i][3]*X[i][4]  
  X_new[i][36]=2*X[i][3]*X[i][5] 
  X_new[i][37]=2*X[i][3]*X[i][6]
  X_new[i][38]=2*X[i][3]*X[i][7]

  X_new[i][39]=2*X[i][4]*X[i][5] 
  X_new[i][40]=2*X[i][4]*X[i][6]
  X_new[i][41]=2*X[i][4]*X[i][7]

  X_new[i][42]=2*X[i][5]*X[i][6]
  X_new[i][43]=2*X[i][5]*X[i][7]

  X_new[i][44]=2*X[i][6]*X[i][7]

  X_new[i][45]=1



ans=[]
for i in range(len(X_new)):
  ans.append(predict(X_new[i]))
      
ans=np.array(ans)
np.savetxt('/content/drive/MyDrive/ECS271/Assignment3/ans_bonus.txt',ans)

In [ ]:
len(ans)

3747